In [23]:
from string import Template
import pandas as pd

import sqlite3 as sql 

from blm_activity_db import BlmActivityDb

In [76]:
period_template = Template("""
## Overview for period $period

| Measure | Activist | Counter-protest | Unknown |
|-----:|:-----:|:-----:|:-----|
| **Accounts** | **$num_accounts_a** | **$num_accounts_c** | **$num_accounts_u** |
| Percent of total | $pct_accounts_a | $pct_accounts_c | $pct_accounts_u |
| Tweets/day | $tweets_day_a | $tweets_day_c | $tweets_day_u |
| Retweets/day | $retweets_day_a |  $retweets_day_c | $retweets_day_u |
| **Communities** | **$num_comms_a** | **$num_comms_c** | **$num_comms_u** |
| Percent of total | $pct_comms_a% | $pct_comms_c% | $pct_comms_u% |
| Largest size | $largest_a | $largest_c | $largest_u |


""")

def period_overview_report(
    period: int,
    num_accounts_a: int,
    num_accounts_c: int,
    num_accounts_u: int,
    tweets_day_a: float,
    tweets_day_c: float,
    tweets_day_u: float,
    retweets_day_a: float,
    retweets_day_c: float,
    retweets_day_u: float,
    num_comms_a: int,
    num_comms_c: int,
    num_comms_u: int,
    largest_a: int,
    largest_c: int,
    largest_u: int,
):
    total_accounts = num_accounts_a + num_accounts_u + num_accounts_c
    total_communities = num_comms_a + num_comms_u + num_comms_c
    subs = {
        "period": period,
        "num_accounts_a": num_accounts_a,
        "num_accounts_c" : num_accounts_c,
        "num_accounts_u": num_accounts_u,
        "pct_accounts_a": round(100 * num_accounts_a / total_accounts, 2),
        "pct_accounts_c": round(100 * num_accounts_c / total_accounts, 2),
        "pct_accounts_u": round(100 * num_accounts_u / total_accounts, 2),
        "tweets_day_a": round(tweets_day_a, 2),
        "tweets_day_c": round(tweets_day_c, 2),
        "tweets_day_u": round(tweets_day_u, 2),
        "retweets_day_a": round(retweets_day_a, 2),
        "retweets_day_c": round(retweets_day_c, 2),
        "retweets_day_u": round(retweets_day_u, 2),
        "num_comms_a": num_comms_a,
        "num_comms_c": num_comms_c,
        "num_comms_u": num_comms_u,
        "pct_comms_a": round(100 * num_comms_a / total_communities, 2),
        "pct_comms_c": round(100 * num_comms_c / total_communities, 2),
        "pct_comms_u": round(100 * num_comms_u / total_communities, 2),
        "largest_a": largest_a,
        "largest_c": largest_c,
        "largest_u": largest_u,
    }
    return period_template.safe_substitute(subs)

In [12]:
db_path = "D:/BLM-db/blm.db"
query = \
"SELECT Stance, COUNT(*) "\
"FROM Community c JOIN AccountActivity aa "\
"ON c.CommunityId = aa.CommunityId "\
"WHERE c.PeriodId = 1 and aa.PeriodId = 1 " \
"GROUP BY c.Stance "\
""
conn = sql.connect(db_path)
with conn:
    cur = conn.cursor()
    cur.execute(query)
    results = cur.fetchall()

for result in results:
    print(result[0], result[1])

-1 606
1 24675


In [85]:
def get_account_stats(period):
    query = \
    "SELECT Stance, COUNT(*), AVG(a.NumTweets), AVG(a.NumRetweets) " \
    "FROM Community c JOIN AccountActivity a " \
    "ON c.CommunityId = a.CommunityId " \
    "WHERE c.PeriodId = ? and a.PeriodId = ? " \
    "GROUP BY Stance " \
    "ORDER BY Stance DESC"
    with conn:
        cur = conn.cursor()
        cur.execute(query, (period, period))
        results = cur.fetchall()
        #cur.execute(query_u, (period,))
        #u_result = cur.fetchone()
    # results = [ca_results[0], ca_results[1]]#, u_result]
    if len(results) == 2: # no unknown communities
        results = [results[0], (0, 0, 0.0, 0.0), results[1]]
    return pd.DataFrame(data=results, columns=["Stance", "NumAccounts", "AvgTweets", "AvgRetweets"])

adf = get_account_stats(3)
print(adf)
adf.NumAccounts[0], adf.NumAccounts[1]

   Stance  NumAccounts  AvgTweets  AvgRetweets
0       1       199286   4.112005     2.681272
1       0          460  19.004348     0.997826
2      -1        14492   3.376484     1.979851


(199286, 460)

In [79]:
def get_community_stats(period):
    max_size_query = \
    "WITH CommunityCounts As ("\
    "SELECT Stance, c.CommunityId, COUNT(*) as Size "\
    "FROM Community c JOIN AccountActivity a "\
    "ON c.CommunityId = a.CommunityId " \
    "WHERE c.PeriodId = ? and a.PeriodId = ? " \
    "GROUP BY c.CommunityId) "\
    "SELECT Stance, MAX(Size) FROM CommunityCounts "\
    "GROUP BY Stance "\
    "ORDER BY Stance DESC"
    num_communities_query = \
    "SELECT Stance, COUNT(*) "\
    "FROM Community "\
    "WHERE PeriodId = ? "\
    "GROUP BY Stance " \
    "ORDER BY Stance DESC"
    with conn:
        cur = conn.cursor()
        cur.execute(max_size_query, (period, period))
        max_size_results = cur.fetchall()
        cur.execute(num_communities_query, (period,))
        num_communities_results = cur.fetchall()
    activist_row = [x for x in num_communities_results[0]] + [max_size_results[0][1]]
    if len(max_size_results) == 2: # first period
        unknown_row = [0, 0, 0]
        counter_row = [x for x in num_communities_results[1]] + [max_size_results[1][1]]
    else:
        unknown_row = [x for x in num_communities_results[1]] + [max_size_results[1][1]]
        counter_row = [x for x in num_communities_results[2]] + [max_size_results[2][1]]
    rows = [activist_row, unknown_row, counter_row]
    return pd.DataFrame(data=rows, columns=["Stance", "NumCommunities", "MaxSize"])

df = get_community_stats(2)
df

,Stance,NumCommunities,MaxSize
0,1,61,25206
1,0,14,11
2,-1,2,9503


In [81]:
report = ""
for period in range(1, 7):
    acct_df = get_account_stats(period)
    comm_df = get_community_stats(period)

    # accounts info
    num_accounts_a, num_accounts_u, num_accounts_c = acct_df.NumAccounts[0], acct_df.NumAccounts[1], acct_df.NumAccounts[2]
    tweets_day_a, tweets_day_u, tweets_day_c = acct_df.AvgTweets[0], acct_df.AvgTweets[1], acct_df.AvgTweets[2]
    retweets_day_a, retweets_day_u, retweets_day_c = acct_df.AvgRetweets[0], acct_df.AvgRetweets[1], acct_df.AvgRetweets[2]
    
    # communities info       
    num_comms_a, num_comms_u, num_comms_c = comm_df.NumCommunities[0], comm_df.NumCommunities[1], comm_df.NumCommunities[2]
    largest_a, largest_u, largest_c = comm_df.MaxSize[0], comm_df.MaxSize[1], comm_df.MaxSize[2]
    report += period_overview_report(
        period,
        num_accounts_a,
        num_accounts_c,
        num_accounts_u,
        tweets_day_a,
        tweets_day_c,
        tweets_day_u,
        retweets_day_a,
        retweets_day_c,
        retweets_day_u,
        num_comms_a,
        num_comms_c,
        num_comms_u,
        largest_a,
        largest_c,
        largest_u
    )
report_path = "D:/BLM/data/Reports/Overview.md"
with open(report_path, "w") as f:
    f.write(report)
